In [11]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
from gensim.corpora import Dictionary

In [12]:
def make_bow(query, articles):
    name = articles.crawling_trg.unique()[0].split(query)[0].strip()

    articles = articles.content.apply(lambda x: x.split()).to_list().copy()
    articles = [[word if not (query in word) else query for word in article] for article in articles]
    articles = [[word if not (name in word) else name for word in article] for article in articles]

    dct = Dictionary(articles)  # fit dictionary
    bow_articles = [dct.doc2bow(article) for article in articles]  # convert corpus to BoW format
    bow_articles = [{k:v for k, v in bow_article} for bow_article in bow_articles]
    
    return name, dct, bow_articles

In [13]:
def id_by_tf_retrieve(query, name, dct, bow_articles, query_th=0, name_th=0):
    query_id = dct.token2id[query]
    query_tfs = []
    for bow_article in bow_articles:
        query_tfs.append(bow_article[query_id])
    query_tfs = np.array(query_tfs)

    name_id = dct.token2id[name]
    name_tfs = []
    for bow_article in bow_articles:
        name_tfs.append(bow_article[name_id])
    name_tfs = np.array(name_tfs)
    
    tf_pairs = [(q, n) for q, n in zip (query_tfs, name_tfs)]
    ids = {}
    for id, tf_pair in enumerate(tf_pairs):
        query_tf, name_tf = tf_pair[0], tf_pair[1]
        if query_tf >= query_th and name_tf >= name_th:
            ids[id] = query_tfs[id] + name_tfs[id]
    if bool(ids) == False:
        return None, None
    
    doc_id = max(ids, key=ids.get)
    tf_score = max(ids.values())

    return doc_id, tf_score

In [14]:
### 모델과 토크나이저를 SentenceTransformer 클래스로 불러옵니다.
pretrained_model_path = "sbert/training_klue_sts_klue-roberta-base-2022-08-17_23-27-13"
model = SentenceTransformer(pretrained_model_path)

### 유사도를 계산하여 인덱스와 스코어를 반환하는 함수입니다.
def get_indices_and_scores(query, news_contents, top_k):
    with torch.no_grad():
        query_embedding = model.encode(query)
        query_embedding = np.expand_dims(query_embedding, axis=0)
        news_embeddings = model.encode(news_contents.content)
        cos_scores = util.pytorch_cos_sim(query_embedding, news_embeddings).squeeze()

        top_k = min(top_k, len(news_contents.content))
        top_k_results = torch.topk(cos_scores, k=top_k)

        scores = top_k_results.values.squeeze()
        indices = top_k_results.indices.squeeze()

    return indices, scores

In [15]:
### 유사도 점수에 기반하여 문서를 찾아옵니다.

def retrieve_docs(query, business_name, crawling_result):
    ### crawling_trg에 business_name 이름이 있는 것과 없는 것을 구분하여 데이터를 나눕니다.
    crawling_result['checker'] = crawling_result.crawling_trg.str.find(business_name)

    bussiness_news = crawling_result[crawling_result.checker > -1].copy().reset_index(drop=True)
    bussiness_news['idx_original'] = range(len(bussiness_news))

    org_news = crawling_result[crawling_result.checker == -1].copy().reset_index(drop=True)
    org_news['idx_original'] = range(len(org_news))

    business_news_contents = bussiness_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]
    org_news_contents = org_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]
    
    
    ### 비지니스명과 쿼리의 term frequency threshold를 충족한 기사 중 term frequency의 합계가 가장 높은 기사를 가지고 옵니다.(중복시 첫 번째)
    name, dct, bow_articles = make_bow(query, business_news_contents)
    doc_id, tf_score = id_by_tf_retrieve(query, name, dct, bow_articles, query_th=1, name_th=3)
    if doc_id == None:
        print(f'{name}: 주목할 만한 기사 없음.')
        print('프로그램을 종료합니다.')
        return None
    else:
        top_of_business_news_contents = business_news_contents.iloc[doc_id].to_list() + list([tf_score])
        top_of_business_news_contents = pd.DataFrame([top_of_business_news_contents], columns=['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link', 'score'])

    ### 기관별로 데이터를 나눕니다.
    org_news_contents_splits = []
    for org in org_news_contents.crawling_trg.unique():
        org_news_contents_split = org_news_contents[org_news_contents.crawling_trg == org].reset_index(drop=True).copy()
        org_news_contents_splits.append(org_news_contents_split)

    ### 각 기관별 수행. 기관명과 쿼리의 term frequency threshold를 충족한 기사 중 term frequency의 합계가 가장 높은 기사를 가지고 옵니다.(중복시 첫 번째)
    tops_of_org_news_contents_splits = []
    for org_news_contents_split in org_news_contents_splits:
        name, dct, bow_articles = make_bow(query, org_news_contents_split)
        doc_id, tf_score = id_by_tf_retrieve(query, name, dct, bow_articles, query_th=2, name_th=3)
        if doc_id == None:
            print(f'{name}: 주목할 만한 기사 없음.')
            print()
            continue
        top_of_org_news_contents_split = org_news_contents_split.iloc[doc_id].to_list() + list([tf_score])
        tops_of_org_news_contents_splits.append(top_of_org_news_contents_split)

    if len(tops_of_org_news_contents_splits) == 0:
        print('검색한 상위 기관에 대하여 주목할 만한 기사 없음.')
        print('프로그램을 종료합니다.')
        return None
    
    tops_of_org_news_contents_splits = pd.DataFrame(tops_of_org_news_contents_splits, columns=['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link', 'score'])
    tops_of_org_news_contents_splits = tops_of_org_news_contents_splits.sort_values('score', ascending=False).reset_index(drop=True).copy()

    ### 가장 높은 점수의 business_name 문서를 쿼리로 하여 
    ### 가장 높은 점수의 기관별 문서 뭉치와 유사도 점수를 계산하고 유사도 점수 상위 5개 문서를 찾아옵니다.
    indices, scores = get_indices_and_scores(top_of_business_news_contents.content.iloc[0], tops_of_org_news_contents_splits, 5)
    result = tops_of_org_news_contents_splits.iloc[list(indices)].copy()
    result['score'] = scores

    return top_of_business_news_contents, tops_of_org_news_contents_splits, result

In [16]:
crawling_result = pd.read_csv('인공지능_문화체육관광부_클라썸_crawled.csv')
query = '인공지능'
business_name = '클라썸'

In [17]:
print('문서간 유사도 검사를 수행합니다.')
top_of_business_news_contents, tops_of_org_news_contents_splits, result = retrieve_docs(query, business_name, crawling_result)

# top_of_business_news_contents.to_csv('top_scored_business_news_for_query.csv', index=False, encoding='utf-8-sig')
# tops_of_org_news_contents_splits.to_csv('list_of_top_scored_org_news_for_query_by_org.csv', index=False, encoding='utf-8-sig')
# result.to_csv('top_5_orgs_and_their_news_for_top_scored_business_news.csv', index=False, encoding='utf-8-sig')

# print('문서간 유사도 검사가 완료되었습니다. 다음 파일을 생성했습니다.')
# print('top_scored_business_news_for_query.csv')
# print('list_of_top_scored_org_news_for_query_by_org.csv')
# print('top_5_orgs_and_their_news_for_top_scored_business_news.csv')

문서간 유사도 검사를 수행합니다.
국립민속박물관: 주목할 만한 기사 없음.

국립아시아문화전당: 주목할 만한 기사 없음.

국립중앙도서관: 주목할 만한 기사 없음.

대한민국역사박물관: 주목할 만한 기사 없음.

한국예술종합학교: 주목할 만한 기사 없음.

해외문화홍보원: 주목할 만한 기사 없음.

게임물관리위원회: 주목할 만한 기사 없음.

사행산업통합감독위원회: 주목할 만한 기사 없음.

세종학당재단: 주목할 만한 기사 없음.

영상물등급위원회: 주목할 만한 기사 없음.

영화진흥위원회: 주목할 만한 기사 없음.

예술의전당: 주목할 만한 기사 없음.

태권도진흥재단: 주목할 만한 기사 없음.

한국문화예술교육진흥원: 주목할 만한 기사 없음.

한국문화예술위원회: 주목할 만한 기사 없음.

한국언론진흥재단: 주목할 만한 기사 없음.

한국저작권위원회: 주목할 만한 기사 없음.

문화재청: 주목할 만한 기사 없음.

국립무형유산원: 주목할 만한 기사 없음.

궁능유적본부: 주목할 만한 기사 없음.

한국전통문화대학교: 주목할 만한 기사 없음.

한국문화재재단: 주목할 만한 기사 없음.



In [21]:
top_of_business_news_contents.to_csv('top_scored_business_news_for_query.csv', index=False, encoding='utf-8-sig')
tops_of_org_news_contents_splits.to_csv('list_of_top_scored_org_news_for_query_by_org.csv', index=False, encoding='utf-8-sig')
result.to_csv('top_5_orgs_and_their_news_for_top_scored_business_news.csv', index=False, encoding='utf-8-sig')

In [22]:
# top_of_business_news_contents
# tops_of_org_news_contents_splits
for idx, row in result.iterrows():
    print(row)
    print()
    print(row.content)
    print()

idx_original                                                   76
crawling_trg                                         국립중앙박물관 인공지능
pubDate                           Fri, 16 Sep 2022 14:50:00 +0900
title                                  [문화소식] 16일 수원서 문화재지킴이 전국대회
content         국립국어원 '인공지능 언어 능력' 평가 대회 국립중앙박물관 어린이박물관, 데굴데굴 ...
link            https://n.news.naver.com/mnews/article/001/001...
score                                                     0.53998
Name: 2, dtype: object

국립국어원 '인공지능 언어 능력' 평가 대회 국립중앙박물관 어린이박물관, 데굴데굴 놀이터' 운영행사 안내 포스터[문화재청 제공. 재판매 및 DB 금지](서울=연합뉴스) 김예나 기자 = ▲ 16일 수원서 문화재지킴이 전국대회 = 문화재청은 한국문화재지킴이단체연합회와 함께 16일 경기 수원 화성 행궁 광장 일대에서 '문화재지킴이 전국대회' 행사를 연다. 문화재지킴이 운동은 국민이 문화재를 자발적으로 가꾸자는 취지로 2005년 시작됐다. 현재 전국에서 6만9천여 명이 문화재 주변 정화, 훼손 방지, 문화재 감시와 홍보 등 다양한 활동을 하고 있다. 전국대회는 이들 지킴이가 한데 모여 활동 공로를 격려하는 자리다. 올해 행사에서는 곳곳의 문화재지킴이 활동을 소개하고 유공자에게 상을 줄 예정이다. 깃발 공연, 국악 공연 등이 열리며 수원 화성을 지켜주는 신을 모신 사당인 '성신사'에서 고유제도 지낼 예정이다. 행사는 문화유산채널 유튜브에서 생중계한다. 행사 안내 포스터[국립국어원 제공. 재판매 및 DB 금지]▲ 국립